Kobert 모델에 대한 설명

In [1]:
!git clone https://github.com/SKTBrain/KoBERT.git

Cloning into 'KoBERT'...
remote: Enumerating objects: 428, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 428 (delta 129), reused 108 (delta 106), pack-reused 275
Receiving objects: 100% (428/428), 219.68 KiB | 3.14 MiB/s, done.
Resolving deltas: 100% (220/220), done.


In [2]:
%cd /content/KoBERT/
!pip install -r requirements.txt

/content/KoBERT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 129 kB 5.1 MB/s 
     |████████████████████████████████| 344 kB 35.3 MB/s 
     |████████████████████████████████| 54.7 MB 1.3 MB/s 
     |████████████████████████████████| 4.5 MB 44.3 MB/s 
     |████████████████████████████████| 1.2 MB 47.2 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:39tcmalloc: large alloc 1147494400 bytes == 0x39ab4000 @  0x7f036362b615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████████████████████| 881.9 MB 16 kB/s 
     |████████████████████████████████| 2.5 MB 37.4 MB/s 
     |████████████████████████████████| 6.7 MB 39.1 MB/s 
     |██████

In [3]:
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, notebook

from torch.nn import init
import gc
import unicodedata
import re

import requests
import pprint
import json

import sys

In [5]:
class BERTDataset(Dataset):
    def __init__(self, data, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = []

        if len(data)<=max_len:
            self.sentences.append(transform([data]))
        else:
            self.sentences.append(transform([data[:max_len]]))


    def __getitem__(self, i):
        return (self.sentences[i])

    def __len__(self):
        return (len(self.sentences))

In [6]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=11,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [7]:
#모델에 넣기 위해 전처리 하는 과정
def GetMediaCategory(text):
    text = unicodedata.normalize('NFC',text)
    text = ' '.join(re.compile('[가-힣]+').findall(text))
    if len(text) == 0:
        text = '기타'

    #datalist = BERTDataset(captionlist,tok, max_len, True, False)
    data = BERTDataset(text, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, num_workers=num_workers)
    gc.collect()
    wholeout=[]
    wholevalue=[]
    # for (token_ids, valid_length, segment_ids) in datalist:
    for batch_id,(token_ids, valid_length, segment_ids) in enumerate(notebook.tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        outlist = []
        valuelist = []
        out = modelbest(token_ids, valid_length, segment_ids)
        for outi in out:
            valuelist.append(outi.max().tolist())
            if outi.max().tolist() > threshold:
                outlist.append(categorylist[outi.argmax()])
            else:
                outlist.append('기타')
        wholeout+=outlist
        wholevalue+=valuelist

    return wholeout, wholevalue

In [8]:
##GPU 사용 시
# there are totally five GPU in server,we can routed to 0:4.
# device = torch.device("cuda:0")
device = torch.device('cpu')

##CPU 사용 시
# device = torch.device('cpu')
gc.collect()

bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

/content/KoBERT/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]
using cached model. /content/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [9]:
# define parameter
max_len = 512
batch_size = 6
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 20
learning_rate =  5e-6  #5e-5  2e-5
num_workers = 2
n_splits = 5
model_name = 'kobertbest_512.pt'

categorylist = ["화장품","패션","요리음식","여행아웃도어","인테리어","엔터테인먼트","육아","아이티","자동차","헬스/피트니스","반려동물"]
threshold = 5.26

modelbest = torch.load("/content/gdrive/MyDrive/Etri/" + model_name, map_location=device)
modelbest.to(device)
modelbest.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [10]:
#제대로 나오는지 확인용
title_classlist, title_valuelist = GetMediaCategory("민카롱 데일리메이크업 웜톤 쿨톤 피부하얘지는법 미백크림 잡티세럼 건성스킨케어 지성스킨케어 스킨케어인생템 토너패드 스킨케어루틴 나이트루틴 꿀피부")
print(title_classlist[0])

  0%|          | 0/1 [00:00<?, ?it/s]

화장품
